In [ ]:
from manim import *

In [ ]:
config.media_width = "100%"
config.verbosity = "WARNING"
config.pixel_width = 600
config.pixel_height = 400

config.background_color = LOGO_BLACK

## All new: Manim's coloring system

Check what Manim's documentation has to say about colors: https://docs.manim.community/en/v0.18.0/reference/manim.utils.color.html

In [ ]:
# all new color implementation!
ManimColor?

In [ ]:
color_inputs = VGroup(
    ManimBanner().set(color=ManimColor("#c0ffee")),  # from hexcode
    ManimBanner().set(color=ManimColor((132, 122, 85))),  # from RGB int tuple (range 0-255)
    ManimBanner().set(color=ManimColor((0.32, 0.35, 0.7))),  # from rgb float tuple (range 0-1)
    ManimBanner().set(color=ManimColor(8435209)),  # from internal integer: 8435209 == 0x80B609 -> "#80B609"
).arrange_in_grid(2, 2, buff=1).scale_to_fit_width(config.frame_width-1)
color_inputs.get_image()

In [ ]:
squares = VGroup(  # careful not to mix int/float up!
    Square(color=ManimColor((1, 1, 1)), fill_opacity=1),
    Square(color=ManimColor((1., 1., 1.)), fill_opacity=1),
).scale(2).arrange(RIGHT)
squares.get_image()

In [ ]:
squares = VGroup(  # arguments to ManimColor can be passed to color-arguments directly:
    Square(color='maroon', fill_opacity=1),
    Square(color='Maroon', fill_opacity=1),
    Square(color='mArOon', fill_opacity=1),
).scale(2).arrange(RIGHT)
squares.get_image()

New: Manim comes with more predefined colors, see their module documentations:
- `manim_colors` [Colors included in the global name space.](https://docs.manim.community/en/v0.18.0/reference/manim.utils.color.manim_colors.html#module-manim.utils.color.manim_colors)
- `AS2700` [Australian Color Standard](https://docs.manim.community/en/v0.18.0/reference/manim.utils.color.AS2700.html#module-manim.utils.color.AS2700)
- `BS381` [British Color Standard](https://docs.manim.community/en/v0.18.0/reference/manim.utils.color.BS381.html#module-manim.utils.color.BS381)
- `XKCD` [Colors from the XKCD Color Name Survey](https://docs.manim.community/en/v0.18.0/reference/manim.utils.color.XKCD.html#module-manim.utils.color.XKCD)
- `X11` [X11 Colors](https://docs.manim.community/en/v0.18.0/reference/manim.utils.color.X11.html#module-manim.utils.color.X11)

In [ ]:
%%manim -qh NewColors

class NewColors(Scene):
    def construct(self):
        self.camera.background_color = '#000000'
        
        xkcd_color_mobjects = VMobject()
        xkcd_colors = [
            color for color_name, color in 
            inspect.getmembers(XKCD, lambda obj: isinstance(obj, ManimColor))
        ]
        
        for color in xkcd_colors:
            col_square = Square()
            col_square.set_fill(color, opacity=1)
            col_square.set_stroke(opacity=0)
            xkcd_color_mobjects.add(col_square)
            
        def color_sort(col):
            h, s, v = col.to_hsv()
            return h, v
            
        xkcd_sorted_ind = sorted(
            range(len(xkcd_colors)),
            key=lambda ind: xkcd_colors[ind].to_hsv()
        )
        
        xkcd_color_mobjects.arrange_in_grid(23, 41)
        xkcd_color_mobjects.width = config.frame_width
        
        for ind, col_square in enumerate(xkcd_color_mobjects):
            col_square.generate_target()
            target_ind = xkcd_sorted_ind.index(ind)
            col_square.target.move_to(xkcd_color_mobjects[target_ind])
        
        
        self.play(FadeIn(xkcd_color_mobjects, lag_ratio=0.001, run_time=5))
        self.wait()
        
        move_squares = AnimationGroup(*[
                MoveToTarget(col_square)
                for col_square in xkcd_color_mobjects
            ],
            lag_ratio=0.001,
            run_time=5,
        )
        self.play(move_squares)
        self.wait(1)
        
        self.play(FadeOut(xkcd_color_mobjects, lag_ratio=0.001, run_time=5))
        
        

## New: Added new module `manim.typing`, plus lots of type hints

Type hints are what allow your IDE to provide useful autocompletion.

New: Several predefined types, and a (start of) complete overhaul of our internal typing guidelines, see https://docs.manim.community/en/v0.18.0/contributing/typings.html

In [ ]:
VMobject?

## New: `manim checkhealth`

In [ ]:
!manim checkhealth --help

## New rate functions: `smoothstep`, `smootherstep`, `smoothererstep`

Documentation incl. list of rate functions: https://docs.manim.community/en/stable/reference/manim.utils.rate_functions.html

In [ ]:
%%manim -qh RateFuncDemo

config.background_color = '#455D3E'

class RateFuncDemo(Scene):
    def construct(self):
        props = [
            ("smooth", X11.SIENNA1),
            ("smoothstep", X11.SIENNA2),
            ("smootherstep", X11.SIENNA3),
            ("smoothererstep", X11.SIENNA4)
        ]
        squares = VGroup()
        for label_text, color in props:
            col_square = Square()
            col_square.set_fill(color, opacity=1)
            col_square.set_stroke(opacity=0)
            label = Text(label_text, font_size=24, font="Monospace")
            label.next_to(col_square, UP)
            col_square.add(label)
            squares.add(col_square)
            
        squares.arrange(RIGHT, buff=1).shift(2*UP)
        self.add(squares)
        
        progress = VDict({
            "line": Line(4*LEFT, 4*RIGHT),
            "marker": Dot(radius=0.15)
        })
        progress["marker"].move_to(progress["line"].point_from_proportion(0))
        progress.next_to(squares, UP)
        self.add(progress)
        
        anims = []
        for ind, (rate_name, _) in enumerate(props):
            rate_func = getattr(rate_functions, rate_name)
            anims.append(squares[ind].animate(run_time=4, rate_func=rate_func).shift(4*DOWN))
        
        self.wait()
        self.play(
            *anims, 
            UpdateFromAlphaFunc(
                progress, 
                lambda mobj, alpha: mobj["marker"].move_to(mobj["line"].point_from_proportion(alpha)),
                run_time=4,
                rate_func=linear,
            )
        )
        self.wait()
        

### ... and several other improvements!

Complete list of changes: https://docs.manim.community/en/v0.18.0/changelog/0.18.0-changelog.html